In [1]:
import re
import json
import requests
import csv
import numpy as np
import pandas as pd
import time
import pymysql

In [2]:
#用get方法访问服务器并提取页面数据
def getHtml(url):
    headers = {
        "User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36",
        "Host": "22.push2.eastmoney.com",
        "Connection": "keep-aliv",
    }

    try:
        r = requests.get(url,headers)
        print(r)
        d = re.compile('"data":({.*)}\);')
        data = re.findall(d,r.text)[0]
        #print(data)
        data1 = json.loads(data)
        #pirnt(data1)
        return data1['diff']
    except Exception as e:
        print(e)
        return e

In [13]:
#获取单个页面股票数据
def getOnePageStock():
    #上海
    page = 1
    url_sh = "http://50.push2.eastmoney.com/api/qt/clist/get?cb=jQuery112405015883081645616_1642902438140&pn="+str(page)+"&pz=20&po=1&np=1&ut=bd1d9ddb04089700cf9c27f6f7426281&fltt=2&invt=2&fid=f3&fs=m:1+t:2,m:1+t:23&fields=f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f12,f13,f14,f15,f16,f17,f18,f20,f21,f23,f24,f25,f22,f11,f62,f128,f136,f115,f152&_=1642902438141"
    
    #自动爬取多页，并在结束时停止
    result_sh = [['股票代码',"地区", '名称']];
    while page <= 107: 
        data = getHtml(url_sh)
        for line in data:
            result_sh.append([line['f12'],line['f13'],line['f14']])     
        print(result_sh)
        print("已加载第"+str(page)+"页")
        page +=1
    with open("shstock.csv","w") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(result_sh)
        
    #深圳        
    page = 1
    result_sz = [['股票代码',"地区", '名称']];
    url_sz = "http://50.push2.eastmoney.com/api/qt/clist/get?cb=jQuery112405015883081645616_1642902438140&pn="+str(page)+"&pz=20&po=1&np=1&ut=bd1d9ddb04089700cf9c27f6f7426281&fltt=2&invt=2&fid=f3&fs=m:0+t:6,m:0+t:80&fields=f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f12,f13,f14,f15,f16,f17,f18,f20,f21,f23,f24,f25,f22,f11,f62,f128,f136,f115,f152&_=1642902438328"    
    #自动爬取多页，并在结束时停止
    while page <= 134: 
        data = getHtml(url_sz)
        for line in data:
            result_sz.append([line['f12'],line['f13'],line['f14']])     
        print(result_sz)
        print("已加载第"+str(page)+"页")
        page +=1
    with open("szstock.csv","w") as csvfile:
            writer = csv.writer(csvfile)
            writer.writerows(result_sz)
    return result_sh,result_sz        
    

In [14]:
def save_all_data(data,city):
   # 保存文件
    daytime = time.strftime("%Y.%m.%d", time.localtime()) 
    with open("stock"+daytime+city+"K.csv","a") as csvfile:
            writer = csv.writer(csvfile)
            writer.writerows(data)


In [17]:
def getStockK(s,city):
    for i in range(0,len(s)):
        time.sleep(1)
        print("正在爬取股票"+str(s[i][2]))
        url = "http://push2his.eastmoney.com/api/qt/stock/kline/get?cb=jQuery112407309929707829328_1642905966239&fields1=f1%2Cf2%2Cf3%2Cf4%2Cf5%2Cf6&fields2=f51%2Cf52%2Cf53%2Cf54%2Cf55%2Cf56%2Cf57%2Cf58%2Cf59%2Cf60%2Cf61&ut=7eea3edcaed734bea9cbfc24409ed989&klt=101&fqt=1&secid="+str(s[i][1])+"."+str(s[i][0]).zfill(6)+"&beg=0&end=20500000&_=1642905966385"
        headers = {
                "User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36",
                "Host": "22.push2.eastmoney.com",
                "Connection": "keep-aliv",
            }
        try:
            r = requests.get(url,headers)
            print(r)
            d = re.compile('"klines":\[.*\]');
            data = re.findall(d,r.text)[0]
            data = data.replace('"klines":["','').replace('"]',','+str(s[i][0])+'').replace('","',','+str(s[i][0])+';')
            data1 = data.split(";");
            data2 = []
            for da in data1:  
                da1 = da.split(",")
                data2.append(da1)
            save_all_data(data2,city)
        except Exception as e:
            print(e)



In [20]:
def main():
    #sh,sz = getOnePageStock()
    sh = pd.read_csv("shstock.csv", encoding="utf8")
    sh = sh.values.tolist()
    sz = pd.read_csv("szstock.csv", encoding="utf8")
    sz = sz.values.tolist()
    headers = [['日期','开盘价','收盘价','最高价','最低价','成交量','成交额','振幅','涨跌幅','涨跌额','换手率','股票编号']]
    daytime = time.strftime("%Y.%m.%d", time.localtime()) 
    with open("stock"+daytime+"shK.csv","w") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(headers)
    getStockK(sh,"sh")
    with open("stock"+daytime+"szK.csv","w") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(headers)
    getStockK(sz,"sz")


In [21]:
main()

正在爬取股票0果麦文化
<Response [200]>
正在爬取股票1安硕信息
<Response [200]>
正在爬取股票2中科信息
<Response [200]>
正在爬取股票3扬电科技
<Response [200]>
正在爬取股票4宜安科技
<Response [200]>
正在爬取股票5皇氏集团
<Response [200]>
正在爬取股票6广博股份
<Response [200]>
正在爬取股票7厦门信达
<Response [200]>
正在爬取股票8华天酒店
<Response [200]>
正在爬取股票9河化股份
<Response [200]>
正在爬取股票10亚联发展
<Response [200]>
正在爬取股票11安妮股份
<Response [200]>
正在爬取股票12新华联


KeyboardInterrupt: 

In [41]:
conn= pymysql.connect(host='127.0.0.1',
                      port=3306,
                      user='root',
                      password="111",
                      charset='utf8')
cursor = conn.cursor()

In [42]:
sqlSentence2 = "use stockDataBase;"
cursor.execute(sqlSentence2)

0

In [43]:
daytime=time.strftime("%Y.%m.%d", time.localtime()) 
fileNamesz=("stock"+daytime+"szK.csv")
data_sh = pd.read_csv(fileNamesz, encoding="utf8")

In [44]:
try:
    sqlSentence3 = "create table stock"  + "(股票代码 VARCHAR(10),名称 VARCHAR(10),日期 date,收盘价 float,涨跌幅 float,涨跌额 float,成交量 bigint,成交额 bigint,振幅 float, 最高价 float,最低价 float,昨日收盘价 float,开盘价 float,量比 float,换手率 float,市盈率 float,市净率 float,总市值 bigint,流通市值 bigint)"
    cursor.execute(sqlSentence3)
except Exception as e:
    print(e)


In [46]:
def sqlsave(data):
    print('正在存储')
    length = len(data)
    for i in range(0, length):
        record = tuple(data.loc[i])
        #插入数据语句
        try:
            sqlSentence4 = "insert into stock" + "(日期,开盘价,收盘价,最高价,最低价,成交量,成交额,振幅,涨跌幅,涨跌额,换手率,股票代码)\
            values ('%s',%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,'%s')" % record
            #获取的表中数据很乱，包含缺失值、Nnone、none等，插入数据库需要处理成空值
            cursor.execute(sqlSentence4)
        except Exception as e:
            print(sqlSentence4)
            #如果以上插入过程出错，跳过这条数据记录，继续往下进行
            print(e)
            break

In [47]:
sqlsave(data_sh)

正在存储


In [48]:
#关闭游标，提交，关闭数据库连接
cursor.close()
conn.commit()
conn.close()
